In [49]:
import pandas as pd

# Merging datasets

We know have different datasets in seperate files. We want to merge them all together into one dataframe to be used for our modeling. During this process, we are bound to lose some data that does not match up with the other datasets. As long as we do not lose too many data points, that will be okay.

# Read in Data

In [50]:
bills = pd.read_csv('../../Data/Bills_Data/state_bills_2017_2018.csv.zip')
abstracts = pd.read_csv('../../Data/Bills_Data/bill_abstracts_2017_2018.csv.zip')
progression = pd.read_csv('../../Data/Legislative_Progression/bill_progression_2017_2018.csv.zip')
codebook = pd.read_csv('../../Data/Legislative_Progression/codebook.csv.zip')
leg_control_2017 = pd.read_csv('../../Data/Legislative_Control_Data/legis_control_2017.csv')
leg_control_2018 = pd.read_csv('../../Data/Legislative_Control_Data/legis_control_2018.csv')
bills_sponsor_party = pd.read_csv('../../Data/Bills_Data/bills_sponsor_party.csv.zip')

/var/folders/2s/0cx_vgq111qdb9jklnsjxt5w0000gp/T/ipykernel_1004/3233608882.py:1: DtypeWarning: Columns (8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  bills = pd.read_csv('../../Data/Bills_Data/state_bills_2017_2018.csv.zip')
/var/folders/2s/0cx_vgq111qdb9jklnsjxt5w0000gp/T/ipykernel_1004/3233608882.py:3: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  progression = pd.read_csv('../../Data/Legislative_Progression/bill_progression_2017_2018.csv.zip')


# Merging bills with bill's progression data

We will need to make a common id that we can merge the datsets on. As both datasets were derived from the common source of Open States, this should not be too difficult. The progression data has a bill_id colum and so does the bills data. The values in this colum look like: "HB 1", "HB 102", "SB 1001" etc. The letters represent where the bill was made and the numbers represent when the bill was made. So, the first bill introduced in the Colorado House for a particular session would be "HB 1", but this is also the case for the first bill introduced in any state as well. All in all, we need more than just the bill identifier to merge the two datasets. 

We will also need to use the session identifier for both datasets. This looks different sate by state but identifies which legislative session the bill was introduced in. Just in case that two states use the same name for a legislative session, for example: "2017 session", we will also make sure the states match up too.

Thus, to merge the two datasets, we will make a custom id for each row that will consist of the State abbreviation, the legisaltive session and the bill id. We will then be able to match both datsets together.

In [51]:
#Dictionary that converts sate names to abbreviations , needed for bills data
us_state_to_abbrev = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
    "New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH","Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", 
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY", 
    "District of Columbia": "DC", "American Samoa": "AS", "Guam": "GU", "Northern Mariana Islands": "MP", "Puerto Rico": "PR", 
    "United States Minor Outlying Islands": "UM", "U.S. Virgin Islands": "VI"
}

In [52]:
#Making custom ID for consisting of state, session name and bill id for the progression dataset
progression['custom_id'] = progression['state'] + progression['sess_str'] + progression['bill_id']

#Formatting bills data to match progression data
bills['jurisdiction'] = bills['jurisdiction'].map(us_state_to_abbrev) #Convert state names into abbreviations
bills['jurisdiction'] = bills['jurisdiction'].map(lambda x: str(x).lower()) #undercase the abbreviations

#Making the custom ID for bills data
bills['custom_id'] = bills['jurisdiction'] +  bills['session_identifier'] + bills['identifier']

In [53]:
df = bills.merge(progression, how = 'inner', on = 'custom_id') #Merge on the custom id and leave out rows that don't match

In [54]:
df.head(1)

,id,identifier,title,classification,subject,session_identifier,jurisdiction,organization_classification,bill_id_x,related_bill_id,...,dum_76_concur,dum_78_intpass,dum_80_intbr,dum_82_veto,dum_83_vetoline,dum_86_vetoover,dum_88_sign,dum_90_law,tn_primary_bill_id,tn_comp_id
0,ocd-bill/f1741c6f-c9fc-4811-8a5f-aca07d1ae90c,SB 53,An Act relating to insurance coverage for cont...,['bill'],"['BOARDS & COMMISSIONS', 'DRUGS', 'HEALTH & SO...",30,ak,upper,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN


In [55]:
print(f'Shape of Bills data: {bills.shape} \nShape of Bill\'s Progression data: {progression.shape}\nShape of merged data: {df.shape}')

Shape of Bills data: (348892, 13) 
Shape of Bill's Progression data: (225251, 41)
Shape of merged data: (159986, 53)


We lossed many rows, but we still have almost 160,000. This will be enough for our study.

# Merging with the legislative control data

For this merge, we will make our own custom id again. It will just consist of the year and state of each row.

In [56]:
#Dictionary that converts sate names to abbreviations , needed to use plotly graph
us_state_to_abbrev = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi ": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
    "New York": "NY","North Carolina": "NC","North Dakota  ": "ND","Ohio": "OH","Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island ": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", 
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY", 
    "District of Columbia": "DC", "American Samoa": "AS", "Guam": "GU", "Northern Mariana Islands": "MP", "Puerto Rico": "PR", 
    "United States Minor Outlying Islands": "UM", "U.S. Virgin Islands": "VI"
}

In [57]:
#Formatting the control data to match progression data for both years
leg_control_2017['states'] = leg_control_2017['states'].map(us_state_to_abbrev) #Convert state names into abbreviations
leg_control_2017['states'] = leg_control_2017['states'].map(lambda x: str(x).lower()) #undercase the abbreviations
leg_control_2017['year'] = 2017 #add year colum

leg_control_2018['states'] = leg_control_2018['states'].map(us_state_to_abbrev) #Convert state names into abbreviations
leg_control_2018['states'] = leg_control_2018['states'].map(lambda x: str(x).lower()) #undercase the abbreviations
leg_control_2018['year'] = 2018 #add year colum

leg_control = pd.concat([leg_control_2017, leg_control_2018]) #Combine both datasets into one

In [58]:
#Make custom id to match up with legislative control data
df['custom_id2'] = df['state'] + df['year1'].map(lambda x: str(x))

#Do the same for control data
leg_control['custom_id2'] = leg_control['states'] + leg_control['year'].map(lambda x: str(x))

In [59]:
df = df.merge(leg_control, how = 'inner', on = 'custom_id2')

In [60]:
df.head(1)

,id,identifier,title,classification,subject,session_identifier,jurisdiction,organization_classification,bill_id_x,related_bill_id,...,tn_primary_bill_id,tn_comp_id,custom_id2,states,senate_dem,senate_rep,house_dem,house_rep,gov_party,year
0,ocd-bill/f1741c6f-c9fc-4811-8a5f-aca07d1ae90c,SB 53,An Act relating to insurance coverage for cont...,['bill'],"['BOARDS & COMMISSIONS', 'DRUGS', 'HEALTH & SO...",30,ak,upper,NaN,NaN,...,NaN,NaN,ak2017,ak,6,14,17,21,Ind,2017


In [61]:
print(f'Shape of merged data: {df.shape}')

Shape of merged data: (159986, 61)


# Merging with abstracts data

Although most bills don't have data with thier abstracts, we will merge in what we have.

In [63]:
added = df.merge(abstracts, how = 'inner', left_on= 'id', right_on= 'bill_id').shape[0]
print(f'we are only addding {added} abstracts to our dataset.')

we are only addding 47200 abstracts to our dataset.


In [64]:
df = df.merge(abstracts, how = 'left', left_on= 'id', right_on= 'bill_id') 

# Merging Legislator Sponsor data

This is a straight forward merge, we just merge on the bill ids.

In [67]:
df_with_sponsor = df.merge(bills_sponsor_party, how = 'inner', left_on = 'id_x', right_on = 'bill_id')

/var/folders/2s/0cx_vgq111qdb9jklnsjxt5w0000gp/T/ipykernel_1004/1276777004.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'bill_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_with_sponsor = df.merge(bills_sponsor_party, how = 'inner', left_on = 'id_x', right_on = 'bill_id')


In [68]:
print(f'Much data was lossed. With the sponsorship data we only have {df_with_sponsor.shape[0]} rows.')
print(f'Without the sponsorship data we have {df.shape[0]} rows.')

Much data was lossed. With the sponsorship data we only have 85525 rows.
Without the sponsorship data we have 159986 rows.


I will save both dataframes in seperate files. 

# Saving Data

In [71]:
df_with_sponsor.to_csv('../../Data/Merged_Data/merged_data_with_sponsorship.csv.zip')
df.to_csv('../../Data/Merged_Data/merged_data.csv.zip')

In [72]:
df

,id_x,identifier,title,classification,subject,session_identifier,jurisdiction,organization_classification,bill_id_x,related_bill_id,...,senate_dem,senate_rep,house_dem,house_rep,gov_party,year,id_y,bill_id,abstract,note
0,ocd-bill/f1741c6f-c9fc-4811-8a5f-aca07d1ae90c,SB 53,An Act relating to insurance coverage for cont...,['bill'],"['BOARDS & COMMISSIONS', 'DRUGS', 'HEALTH & SO...",30,ak,upper,NaN,NaN,...,6,14,17,21,Ind,2017,NaN,NaN,NaN,NaN
1,ocd-bill/fc02f0e2-a789-48e2-bb71-6839db4af4a1,SB 33,An Act naming the state ferries built in Ketch...,['bill'],"['BOATS & BOATING', 'MARINE HIGHWAY', 'TRANSPO...",30,ak,upper,NaN,NaN,...,6,14,17,21,Ind,2017,NaN,NaN,NaN,NaN
2,ocd-bill/995b8a0b-41dd-4918-a1ae-ab32c7b41070,HB 141,An Act relating to allocations of funding for ...,['bill'],"['BUSINESS', 'EDUCATION', 'EMPLOYMENT', 'LABOR...",30,ak,lower,NaN,NaN,...,6,14,17,21,Ind,2017,NaN,NaN,NaN,NaN
3,ocd-bill/c1da3d60-e3e7-4c3a-a244-31b137690e2c,SB 103,An Act establishing the Alaska education innov...,['bill'],"['BOARDS & COMMISSIONS', 'COMMUNITY COLLEGES',...",30,ak,upper,NaN,NaN,...,6,14,17,21,Ind,2017,NaN,NaN,NaN,NaN
4,ocd-bill/08bfc0da-90ef-47c0-872b-88701a4f8eaa,HB 77,An Act making corrective amendments to the Ala...,['bill'],"['AIRPORTS', 'APPROPRIATIONS', 'AVIATION', 'BO...",30,ak,lower,NaN,NaN,...,6,14,17,21,Ind,2017,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159981,ocd-bill/cc8d9bb3-064e-4e14-9cf5-d1ee9c26cfab,HB 71,Child custody.,['bill'],[],2018,wy,lower,NaN,NaN,...,3,27,9,51,Rep,2018,NaN,NaN,NaN,NaN
159982,ocd-bill/36472695-f73c-4d90-b773-7a3091b96fb2,SF 85,Certificate of nonviable birth.,['bill'],[],2018,wy,upper,NaN,NaN,...,3,27,9,51,Rep,2018,9117a636-88a7-4dd5-bd48-20f7302e380c,ocd-bill/36472695-f73c-4d90-b773-7a3091b96fb2,AN ACT relating to vital records; permitting r...,description
159983,ocd-bill/1eb952f3-88fc-4878-846f-b1b4a9a9f1c9,HB 24,Investment of public funds-fixed interest bear...,['bill'],[],2018,wy,lower,NaN,NaN,...,3,27,9,51,Rep,2018,NaN,NaN,NaN,NaN
159984,ocd-bill/f9474159-8e7c-4b5f-ad1f-d04ec6a6b004,HB 181,Wage garnishment amendments.,['bill'],[],2018,wy,lower,NaN,NaN,...,3,27,9,51,Rep,2018,NaN,NaN,NaN,NaN
